## Counterfactual Explanation on Structured Data
### CHAPTER 02 - *Model Explainability Methods*

From **Applied Machine Learning Explainability Techniques** by [**Aditya Bhattacharya**](https://www.linkedin.com/in/aditya-bhattacharya-b59155b6/), published by **Packt**

### Objective

In this notebook, we will try to implement some of the concepts related to Counterfactual explanations part of the Example based explainability methods discussed in Chapter 2 - Model Explainability Methods

### Installing the modules

Install the following libraries in Google Colab or your local environment, if not already installed.

In [ ]:
!pip install --upgrade pandas numpy scikit-learn tensorflow dice-ml alibi

### Loading the modules

In [195]:
import dice_ml as dice
import alibi
from dice_ml.utils import helpers as utils  # using helper functions as utility functions
from alibi.explainers import CounterfactualProto, Counterfactual
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs 
from tensorflow.keras.layers import Dense, Input, Embedding, Concatenate, Reshape, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

### Loading the data

For the purpose of simplicity of understanding, we will use some of the standard examples provided in the original repositories for the frameworks DiCE and Alibi. The datasets used will be derived and transformed datasets from original datasets. The sources of the original datasets will be mentioned and I would strongly recommend to look at the original data for more details on the data description and for a more detailed analysis.

In [31]:
data = utils.load_adult_income_dataset()
data.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [10]:
data_description = utils.get_adult_data_info()
data_description

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [13]:
data.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation', 'race',
       'gender', 'hours_per_week', 'income'],
      dtype='object')

In [14]:
data.shape

(26048, 9)

### About the data

**Adult Data Set - UCI Machine Learning Repository**

This dataset is also known as the *Census Income* dataset which is used to predict whether the income exceeds $50k/year based on census data. It is a multivariate dataset used for classification based problems containing 14 different features. More details about this data can be found at - [https://archive.ics.uci.edu/ml/datasets/adult](https://archive.ics.uci.edu/ml/datasets/adult)

### Using the DiCE framework for Counterfactual Explanations

Since the goal of this notebook is to introduce and briefly show usage of certain key frameworks in the context of example based model explainability, I will not focus on importance steps of an end to end ML workflow like EDA, Feature Engineering, Model Training and Evaluation. Mostly I will be using pretained models to demonstrate the generation and working of counterfactual explanations. The DiCE framework will be covered in more details in *Chapter 09 - Other popular XAI frameworks*.

In [29]:
# Preparing the DiCE data object
data_object = dice.Data(dataframe = data,
                           continuous_features = ['age', 'hours_per_week'],
                           outcome_name = 'income'
                          )
# Loading pre-trained models using the DiCE framework
model = utils.get_adult_income_modelpath()
model_object = dice.Model(model_path = model, backend='TF2') # Creating Tensorflow 2.0 model object

In [48]:
# Creating DiCE explanation instance
dice_explanation = dice.Dice(data_object, model_object, method = 'random')

In [49]:
# Let's take a query input to generate Counterfactual Explanations
test_query = {'age':28,
    'workclass':'Self-Employed',
    'education':'HS-grad',
    'marital_status':'Single',
    'occupation':'Service',
    'race': 'Other',
    'gender':'Male',
    'hours_per_week': 45}

In [51]:
# let's generate counterfactual examples
dice_exp = dice_explanation.generate_counterfactuals(test_query,
                                                     total_CFs=3,
                                                     desired_class="opposite",
                                                     features_to_vary=["education", "occupation", "workclass", "marital_status", "hours_per_week"]
                                                    )

# And visualize counterfactual explanation
dice_exp.visualize_as_dataframe(show_only_changes=True, display_sparse_df=False)

Diverse Counterfactuals found! total time taken: 00 min 50 sec
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28.0,Self-Employed,HS-grad,Single,Service,Other,Male,45.0,0.082



Diverse Counterfactual set without sparsity correction (new outcome:  1.0


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Doctorate,Married,White-Collar,-,-,-,0.886
1,-,-,Masters,Married,-,-,-,36.0,0.775
2,-,-,Prof-school,Married,-,-,-,56.0,0.868


From the above observation, we can clearly observe the counterfactual examples to increase income beyond $50K/year. Although we can see that some of the examples can be contradictory and there is not quantitative way to evaluate the best example(which is a major disadvantage of this algorithm). But we can clearly comment that if the person decides to go for higher eductaion or increase his work rate he can get a higher pay. Also, usually it is observed that when a person gets married, the annual income also increases. This is a correlation which has been observed from the data.

Overall, it is very easy to implement this approach and understand the explanations provided by this approach.

### Using the Alibi for Counterfactual Explanations

Now, let us see how to use the Alibi dataset for generating counterfactual examples. We would need to generate a model first, which is supported by the Alibi framework. The Alibi framework will be covered in more details in *Chapter 09 - Other popular XAI frameworks*.

In [71]:
# Dividing data into features and target values
target = data['income']
features = data.iloc[:,:-1]

In [76]:
# Since the dataset consists of many categorical features, we may need to encode these using One-Hot Encoding
def encode_features(df, features):
    '''
    Method for one-hot encoding all selected categorical fields
    '''
    for f in features:
        if(f in df.columns):
            encoded = pd.get_dummies(df[f])
            encoded = encoded.add_prefix(f + '_')
            df = pd.concat([df, encoded], axis=1)
        else:
            print('Feature not found')
            return df
        
    df.drop(columns=features, inplace = True)
    
    return df

In [75]:
features_to_encode = ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
encoded = encode_features(features, features_to_encode)
features = encoded.copy()
features.head()

,age,hours_per_week,workclass_Government,workclass_Other/Unknown,workclass_Private,workclass_Self-Employed,education_Assoc,education_Bachelors,education_Doctorate,education_HS-grad,...,occupation_Blue-Collar,occupation_Other/Unknown,occupation_Professional,occupation_Sales,occupation_Service,occupation_White-Collar,race_Other,race_White,gender_Female,gender_Male
0,28,60,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,1,1,0
1,30,65,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,1,0,1
2,32,50,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,20,35,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
4,41,50,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [178]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=100) # 80% training and 20% test

In [182]:
def model():
    inp = Input(shape=(X_train.shape[1],))
    x = Dense(40, activation='relu')(inp)
    x = Dense(40, activation='relu')(x)
    op = Dense(2, activation='softmax')(x)
    model = Model(inputs=inp, outputs=op)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [183]:
model = model()
model.fit(X_train, to_categorical(y_train), batch_size=64, epochs=200, verbose=0)

In [187]:
# Evaluate the trained model
model.evaluate(X_test, to_categorical(y_test))[1]

0.8403071

In [188]:
query_instance = X_test.iloc[1].values.reshape((1,) + X_test.iloc[1].shape)

In [189]:
# Using Alibi's Counterfactual Prototype method to initialize explainer, fit and generate counterfactual
cf = CounterfactualProto(model, 
                         query_instance.shape, 
                         use_kdtree=True, 
                         theta=10., 
                         max_iterations=1000,
                         c_init=1., 
                         c_steps=10)
# Fit on the training data
cf.fit(X_train.values)
# Form explanation on the query data
explanation = cf.explain(query_instance)

No encoder specified. Using k-d trees to represent class prototypes.


In [190]:
print(f'Original prediction: {explanation.orig_class}')
print('Counterfactual prediction: {}'.format(explanation.cf['class']))

Original prediction: 0
Counterfactual prediction: 1


In [196]:
df_cfe = pd.DataFrame(counterfactual, columns = features.columns)
df_cfe

,age,hours_per_week,workclass_Government,workclass_Other/Unknown,workclass_Private,workclass_Self-Employed,education_Assoc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Prof-school,education_School,education_Some-college,marital_status_Divorced,marital_status_Married,marital_status_Separated,marital_status_Single,marital_status_Widowed,occupation_Blue-Collar,occupation_Other/Unknown,occupation_Professional,occupation_Sales,occupation_Service,occupation_White-Collar,race_Other,race_White,gender_Female,gender_Male
0,28,24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [197]:
df_query = pd.DataFrame(query_instance, columns = features.columns)
df_query

,age,hours_per_week,workclass_Government,workclass_Other/Unknown,workclass_Private,workclass_Self-Employed,education_Assoc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Prof-school,education_School,education_Some-college,marital_status_Divorced,marital_status_Married,marital_status_Separated,marital_status_Single,marital_status_Widowed,occupation_Blue-Collar,occupation_Other/Unknown,occupation_Professional,occupation_Sales,occupation_Service,occupation_White-Collar,race_Other,race_White,gender_Female,gender_Male
0,26,24,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0


In [192]:
# Now let's see the counterfactual example
counterfactual = explanation.cf['X'].astype(int)
change = counterfactual - query_instance
for i, feature_names in enumerate(features.columns):
    if change[0][i] != 0:
        print(f"The feature {feature_names} has to change by {change[0][i]}")

The feature age has to change by 2
The feature education_Some-college has to change by -1
The feature marital_status_Single has to change by -1
The feature occupation_Service has to change by -1


From above observation, we can understand that for the income of the person to increase beyond $50k/year, the person needs to wait for another 2 years, or do a higher education. As observed with the DiCE framework, because of the correlation with marriage and net income from the dataset, the explanation method is suggesting the person to get married. The method is also suggesting to change the occupation type.

### Reference

1. [Diverse Counterfactual Explanations (DiCE) for ML](https://github.com/interpretml/DiCE) - Ramaravind K. Mothilal, Amit Sharma, and Chenhao Tan (2020). Explaining machine learning classifiers through diverse counterfactual explanations. *Proceedings of the 2020 Conference on Fairness, Accountability, and Transparency.*
2. [Alibi](https://github.com/SeldonIO/alibi) - Klaise et. al - Alibi Explain: Algorithms for Explaining Machine Learning Models
3. Adult Income Dataset from UCI Machine Learning Repository - https://archive.ics.uci.edu/ml/datasets/adult
4. Some of the utility functions are taken from the GitHub Repository of the author - https://github.com/adib0073